In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
from pandas import DataFrame,Series
import matplotlib.pyplot as plt
import numpy as np
import re

# Pandas私房手册-常见问题解决方案收集

#### 使用`to_string`方法优化`Series`和`dataframe`的输出

`dataframe`和`Series`对象有一个`to_string`方法，可以优化其输出的显示，比如可以通过`na_rep`参数指定用什么符号来代替默认的`NaN`进行显示：

In [9]:
s = Series([1, np.nan, 2])
print(s.to_string(na_rep=''))

0    1.0
1       
2    2.0


#### 求`dataframe`整个表中的最大值

`dataframe`的`max()`方法只能求出每一列的最大值，现在要求整个表的最大值，如果整个表都是数值类型，那么可以这样：

In [13]:
df = DataFrame(np.random.randint(1, 13, size=(4, 3)))
# 方法1：
df.max().max()
# 方法2：
# df.values.max()
# 方法3：
# df.to_numpy().max()

10

如果又有数值列又有其它不能比较的数据类型的列，可以利用`select_dtypes`方法先选择数值列：

In [34]:
df = DataFrame([[1, 11, 'c'], [2, 2, 'a'], [3, 4, 'd']])
df.select_dtypes(np.number).values.max()

11

如果有数据类型是`object`的列，但是列里又包含数字：

In [70]:
df = DataFrame([[1, 'd', 'c'], [11, 12, 'z'], [3, 4, 8]])
# 先通过applymap判断是否数字，并构造一个mask，将所有非数字设置为负无穷大，然后再求最大值
df.where(df.applymap(lambda x: isinstance(x, (int, float))),
         float('-inf')).max().max()

12.0

#### 将Excel文件的所有sheet读到一个`DataFrame`里

下面的函数可以将所有的sheet读到一个`DataFrame`里，另外，还可以指定一个sheet名称的正则表达式模式，只有符合模式的sheet才会读取：

In [19]:
def concat_excel_sheets(xfile, sheet_pat=None, **kwargs):
    """
    将一个excel文件的所有sheet合并到一个dataframe里，可以根据正则表达式的模式匹配选取
    要合并的sheet。
    
    参数
    ----------
    xfile: 字符串
        要读取的excel文件的文件名
    sheet_pat: 字符串或者正则表达式的模式，默认为None
        用来匹配sheet名称，匹配成功则读取
    kwargs:
        其它所有关键字参数都将传给pandas的read_excel函数
        
    返回
    ----------
    dataframe对象
    """
    sheets = []
    with pd.ExcelFile(xfile) as xls:
        if sheet_pat is not None:
            for sheet in xls.sheet_names:
                if isinstance(sheet_pat, str):
                    pat = re.compile(sheet_pat)
                if pat.match(sheet):
                    sheets.append(sheet)
        else:
            sheets = xls.sheet_names
        if sheets == []:
            raise ValueError("没有可以合并的sheet！")
        df = pd.concat(
            [pd.read_excel(xls, sheet, **kwargs) for sheet in sheets],
            sort=True)
    return df

In [20]:
concat_excel_sheets('excel_demo.xlsx', r'Sheet')

,Unnamed: 0,col1,col2
0,row1,1,2
1,row2,3,4
0,row1,5,6
1,row2,7,8


#### 将列表扩展称为行

pandas有一个expolode方法可以实现：

In [4]:
df = DataFrame({'a':[[1,2,3],[4,5,6]], 'b':[1,2]})
df

,a,b
0,"[1, 2, 3]",1
1,"[4, 5, 6]",2


In [5]:
df.explode('a')

,a,b
0,1,1
0,2,1
0,3,1
1,4,2
1,5,2
1,6,2


但是多行都是列表就不行了，比如：

In [9]:
df = DataFrame({'a':[[1,2],[4,5]], 'b':[1,2], 'c':[[1,2], [3, 4]]})
df

,a,b,c
0,"[1, 2]",1,"[1, 2]"
1,"[4, 5]",2,"[3, 4]"


如果`a`，`c`的列表大小是一样的，那么可以先把非列表的列转换为`index`，然后利用`apply`方法：

In [10]:
df.set_index('b').apply(pd.Series.explode).reset_index()

,b,a,c
0,1,1,1
1,1,2,2
2,2,4,3
3,2,5,4


注意，此时不能有`nan`，否则会报错。

#### 使用空列表填补缺失值

`fillna`可以填补缺失值，但是只能接受标量或者字典，不能接受列表：

In [43]:
df = DataFrame({'a': [np.nan, 2, 3]})
df

,a
0,NaN
1,2.0
2,3.0


In [44]:
try:
    df['a'].fillna([])
except TypeError as e:
    print(e)

"value" parameter must be a scalar or dict, but you passed a "list"


此时可以使用`apply`或者`map`方法：

In [45]:
df['a'].map(lambda x: [] if pd.isna(x) else x)

0     []
1    2.0
2    3.0
Name: a, dtype: object

#### 对值是列表的列进行分列

如果列是字符串，可以使用`Serias.str.split`的方法或者`extract`方法，可是如果是列表，可以借用构造函数和`to_list`方法：

In [3]:
df = DataFrame({'a':[[1,2,3],[4,5,6]], 'b':[1,2]})
df

,a,b
0,"[1, 2, 3]",1
1,"[4, 5, 6]",2


In [59]:
df[['a1', 'a2', 'a3']] = DataFrame(df['a'].to_list())
df

,a,b,a1,a2,a3
0,"[1, 2, 3]",1,1,2,3
1,"[4, 5, 6]",2,4,5,6


注意，`to_list`以后再构造一个`DataFrame`，此时`index`是顺序的序列，如果和原来的`index`不一样，需要在构造函数里面添加`index=df.index`参数。另外，此时也不能包含`nan`，否则也会报错。

In [6]:
DataFrame(df['a'].to_list(), index=df.index)

,0,1,2
0,1,2,3
1,4,5,6
